In [1]:
import os
os.chdir(os.path.dirname(os.getcwd()))

import pandas as pd

In [2]:
data_path = "./data/fact_train_test.csv"
data = pd.read_csv(data_path, sep=";", decimal=",", encoding="windows-1251")
data

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count
0,2012-07-01,1,5,1,328,1193,30252,13005,10036,71.0,1
1,2012-10-01,1,5,1,328,1193,30252,11376,10036,210.0,3
2,2014-03-01,0,1,1,328,3472,30252,29548,10036,67.0,1
3,2014-03-01,0,1,1,328,3472,30252,29158,10036,67.0,1
4,2014-03-01,0,2,1,328,3472,30252,27484,10036,66.0,1
...,...,...,...,...,...,...,...,...,...,...,...
3559227,2023-03-01,1,5,2,1346,1482,31438,31482,0,69.0,1
3559228,2023-03-01,1,5,2,1346,1492,31438,36091,27275,70.0,1
3559229,2023-03-01,1,5,2,1346,1492,31438,35450,27275,70.0,1
3559230,2023-03-01,1,5,2,1346,1492,31438,31482,0,207.0,3


In [3]:
data_grouped = data.groupby(["client_sap_id", "freight_id", "sender_station_id", "recipient_station_id", "sender_organisation_id"]).agg(list)
data_grouped

period  \
client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id                                                      
-1            0          1576              37851                0                                                            [2017-05-01]   
                                           38902                0                                                            [2017-06-01]   
                         2977              38567                7152                                                         [2019-06-01]   
                         3801              26930                7152                                                         [2018-11-01]   
                         3845              28207                8535                                                         [2018-09-01]   
...                                                                                                                                   ...   
 2416         2120       27484             38641                12065                   [2018-12-01, 2019-06-01, 2019-08-01, 2019-10-0...   
                                                                27353                    [2022-01-01, 2022-03-01, 2022-10-01, 2023-02-01]   
 2417         349        38760             38966                4631                                                         [2017-01-01]   
                                                                14971                   [2017-05-01, 2018-05-01, 2018-05-01, 2018-05-0...   
              388        38760             38966                14971                   [2019-03-01, 2019-04-01, 2019-06-01, 2019-06-0...   

                                                                                                                                      rps  \
client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id                                                      
-1            0          1576              37851                0                                                                     [1]   
                                           38902                0                                                                     [1]   
                         2977              38567                7152                                                                  [0]   
                         3801              26930                7152                                                                  [0]   
                         3845              28207                8535                                                                  [0]   
...                                                                                                                                   ...   
 2416         2120       27484             38641                12065                                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   
                                                                27353                                                        [0, 0, 0, 0]   
 2417         349        38760             38966                4631                                                                  [0]   
                                                                14971                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
              388        38760             38966                14971                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]   

                                                                                                                                   podrod  \
client_sap_id freight_id sender_station_id recipient_station_id sender_organisation_id                                                      
-1            0          1576              37851                0                                                                     [5]   
                                        

In [5]:
data_grouped.columns.tolist()

['period', 'rps', 'podrod', 'filial', 'real_weight', 'real_wagon_count']

In [4]:
def train_val_split(periods):
    if len(periods) >= 5:
        return [False] * (len(periods) - 2) + [True] * 2
    else:
        return [False] * len(periods)

In [5]:
data_grouped["is_test"] = data_grouped["period"].apply(train_val_split)

In [6]:
train_val = data_grouped.explode(["period", "rps", "podrod", "filial", "real_weight", "real_wagon_count", "is_test"]).reset_index()

In [7]:
train_val["is_test"].value_counts()[True] / train_val.shape[0]

0.09848191969503534

In [8]:
pd.concat([data, train_val.drop(columns=["is_test"])]).drop_duplicates(keep=False)

,period,rps,podrod,filial,client_sap_id,freight_id,sender_station_id,recipient_station_id,sender_organisation_id,real_weight,real_wagon_count
